In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [3]:
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True
MAXLEN = DIGITS + 1 + DIGITS

In [4]:
# Instantiate a class to generate data
class CharacterTable:

    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax = True):
        if calc_argmax:
            x = x.argmax(axis = -1)
        return  ''.join(self.indices_char[x] for x in x)

In [5]:
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


In [11]:
# The Vectorization process
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype = np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype = np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS +1)

indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [28]:
# Building the model
num_layers = 1

model = keras.Sequential()
model.add(layers.LSTM(16, input_shape = (MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(num_layers):
    model.add(layers.LSTM(16, return_sequences = True))
model.add(layers.Dense(len(chars), activation = 'softmax'))
optimizer = keras.optimizers.Adagrad(learning_rate = 0.75)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer , metrics = ['accuracy'], )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 16)                1856      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 16)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 16)             2112      
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             204       
Total params: 4,172
Trainable params: 4,172
Non-trainable params: 0
_________________________________________________________________


In [29]:
epochs = 10
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
1407/1407 [==============================] - 10s 5ms/step - loss: 1.9400 - accuracy: 0.3210 - val_loss: 1.5550 - val_accuracy: 0.4128
Q 42+383  T 425  ☒ 403 
Q 875+62  T 937  ☒ 903 
Q 176+2   T 178  ☒ 119 
Q 496+452 T 948  ☒ 703 
Q 1+758   T 759  ☒ 761 
Q 441+35  T 476  ☒ 461 
Q 63+132  T 195  ☒ 403 
Q 1+573   T 574  ☒ 461 
Q 224+813 T 1037 ☒ 103 
Q 274+901 T 1175 ☒ 107 

Iteration 2
1407/1407 [==============================] - 6s 4ms/step - loss: 1.3286 - accuracy: 0.5031 - val_loss: 1.3277 - val_accuracy: 0.5048
Q 166+533 T 699  ☒ 611 
Q 97+599  T 696  ☒ 741 
Q 634+96  T 730  ☒ 711 
Q 509+55  T 564  ☒ 514 
Q 600+695 T 1295 ☒ 1444
Q 222+9   T 231  ☒ 211 
Q 2+815   T 817  ☒ 811 
Q 8+626   T 634  ☒ 611 
Q 408+753 T 1161 ☒ 1243
Q 237+91  T 328  ☒ 311 

Iteration 3
1407/1407 [==============================] - 6s 4ms/step - loss: 1.1141 - accuracy: 0.5769 - val_loss: 1.0566 - val_accuracy: 0.6012
Q 200+82  T 282  ☒ 277 
Q 225+766 T 991  ☒ 105 
Q 238+9   T 247  ☑ 247 
Q 35+33  

In [25]:
keras.backend.clear_session()